In [1]:
import pandas as pd
from lxml import html
import numpy as np
#from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
#import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
#%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
#stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and 'http:' not in word]

    return ' '.join(words)

In [2]:
from textblob import TextBlob

In [3]:
train_data = pd.read_csv('train_sentiment_ttk.tsv', sep='\t')
test_data = pd.read_csv('test_sentiment_ttk.tsv', sep='\t')

In [4]:
train_data.head()

,label,text
0,0,"@mkomov Максим, Вашем письмо мы получили. Наши..."
1,0,«Мегафон» стал владельцем 50% акций «Евросети»
2,-1,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...
3,1,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...
4,-1,"@parfenov1960 потому что МТС достало, а пчел н..."


In [5]:
test_data.head()

,label,text
0,-1,RT vzglyad: По делу о работе МТС в Узбекистане...
1,0,RT @kevinuyatukox: http://t.co/ljtrjq91v3 #Кре...
2,0,#Оформить кредитную карту в банке мтс http://t...
3,0,#Как перевести деньги с билайна на кредитную к...
4,0,#Начальник отдела кредитного контроля оао мтс ...


In [6]:
from copy import copy

In [7]:
data = pd.DataFrame()

In [8]:
data['text'] = copy(train_data['text'])

In [9]:
data.head()

,text
0,"@mkomov Максим, Вашем письмо мы получили. Наши..."
1,«Мегафон» стал владельцем 50% акций «Евросети»
2,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...
3,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...
4,"@parfenov1960 потому что МТС достало, а пчел н..."


In [10]:
t = [TextBlob(i) for i in train_data['text']]
data['TextBlob'] = t

In [11]:
data.head()

,text,TextBlob
0,"@mkomov Максим, Вашем письмо мы получили. Наши...","(@, m, k, o, m, o, v, , М, а, к, с, и, м, ,, ..."
1,«Мегафон» стал владельцем 50% акций «Евросети»,"(«, М, е, г, а, ф, о, н, », , с, т, а, л, , ..."
2,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...,"(R, T, , @, f, u, c, k, k, i, e, v, :, , “, ..."
3,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...,"(В, И, Д, Е, О, :, , h, t, t, p, :, /, /, t, ..."
4,"@parfenov1960 потому что МТС достало, а пчел н...","(@, p, a, r, f, e, n, o, v, 1, 9, 6, 0, , п, ..."


In [12]:
s = [i.sentiment for i in data['TextBlob']]
data['sentiments'] = s

In [13]:
data.head()

,text,TextBlob,sentiments
0,"@mkomov Максим, Вашем письмо мы получили. Наши...","(@, m, k, o, m, o, v, , М, а, к, с, и, м, ,, ...","(0.0, 0.0)"
1,«Мегафон» стал владельцем 50% акций «Евросети»,"(«, М, е, г, а, ф, о, н, », , с, т, а, л, , ...","(0.0, 0.0)"
2,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...,"(R, T, , @, f, u, c, k, k, i, e, v, :, , “, ...","(0.0, 0.0)"
3,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...,"(В, И, Д, Е, О, :, , h, t, t, p, :, /, /, t, ...","(0.0, 0.0)"
4,"@parfenov1960 потому что МТС достало, а пчел н...","(@, p, a, r, f, e, n, o, v, 1, 9, 6, 0, , п, ...","(0.0, 0.0)"


In [14]:
for ind, sent in enumerate(data['sentiments']):
    if sent.polarity != 0.0 or sent.subjectivity != 0.0:
        print(data['text'][ind])
        print(sent)
        print('----------------------------------------------------')

#Траффик на Билайне уходит неумолимо :(
#Билайн #Москва http://t.co/KGRzUG10xO
Sentiment(polarity=-0.75, subjectivity=1.0)
----------------------------------------------------
RT @_Beeline_kz: Сегодня День Системного Администратора! =) С праздником, друзья! http://t.co/KiDNwq11ht
Sentiment(polarity=0.625, subjectivity=1.0)
----------------------------------------------------
10 июля 2014 года вещание телеканала CTC Love стартовало в цифровой кабельной сети «Билайн».
Sentiment(polarity=0.5, subjectivity=0.6)
----------------------------------------------------
что-то опять не работает автоплатеж в мегафон :-(
Sentiment(polarity=-0.75, subjectivity=1.0)
----------------------------------------------------
@Wild_Ma13 вся семья на билайне...так бы поменял:)
Sentiment(polarity=0.5, subjectivity=1.0)
----------------------------------------------------
RT @MegaFonUral: Уже завтра мы открываем в Тюмени парк спорта и инноваций «МегаZone».Ждем вас с 12 до 18 на ул. Ленина /Орджоникидзе:) http…


У нас есть 3 класса твитов. Можно попробовать вывести формулу, по которой мы будем определять класс твитов по анализу TextBlob и посчитать f1 меру, потому что accuracy не подойдет.

In [15]:
train_data['label'].value_counts()


 0    4590
-1    2388
 1    1230
Name: label, dtype: int64

In [43]:
test_data['label'].value_counts()

 0    972
-1    902
 1    180
Name: label, dtype: int64

Кажется, что лучше учитывать только polarity, потому что не очень понятно, как нам тут поможет subjectivity. Давайте будем считать, что:
- от -1 до -0.25 - -1
- от -0.25 до 0.25 - 0
- от 0.25 до 1 - 1

In [16]:
pred = []
for ind, sent in enumerate(data['sentiments']):
    if sent.polarity < -0.25:
        pred.append(-1)
    elif sent.polarity > 0.25:
        pred.append(1)
    else:
        pred.append(0)

In [17]:
data['pred'] = pred
data['label'] = copy(train_data['label'])

In [18]:
data.head()

,text,TextBlob,sentiments,pred,label
0,"@mkomov Максим, Вашем письмо мы получили. Наши...","(@, m, k, o, m, o, v, , М, а, к, с, и, м, ,, ...","(0.0, 0.0)",0,0
1,«Мегафон» стал владельцем 50% акций «Евросети»,"(«, М, е, г, а, ф, о, н, », , с, т, а, л, , ...","(0.0, 0.0)",0,0
2,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...,"(R, T, , @, f, u, c, k, k, i, e, v, :, , “, ...","(0.0, 0.0)",0,-1
3,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...,"(В, И, Д, Е, О, :, , h, t, t, p, :, /, /, t, ...","(0.0, 0.0)",0,1
4,"@parfenov1960 потому что МТС достало, а пчел н...","(@, p, a, r, f, e, n, o, v, 1, 9, 6, 0, , п, ...","(0.0, 0.0)",0,-1


In [19]:
print(classification_report(data['label'], data['pred']))

              precision    recall  f1-score   support

          -1       0.69      0.01      0.03      2388
           0       0.56      0.98      0.72      4590
           1       0.37      0.05      0.09      1230

   micro avg       0.56      0.56      0.56      8208
   macro avg       0.54      0.35      0.28      8208
weighted avg       0.57      0.56      0.42      8208



Мы получили хороший результат для нейтрального класса, но ужасный для остальных.
Можно попробовать предобработать текст и запустить еще раз.

In [20]:
data_tok = pd.DataFrame()
data_tok['label'] = copy(data['label'])
data_tok['text'] = copy(data['text'])

In [21]:
data_tok.head()

,label,text
0,0,"@mkomov Максим, Вашем письмо мы получили. Наши..."
1,0,«Мегафон» стал владельцем 50% акций «Евросети»
2,-1,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...
3,1,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...
4,-1,"@parfenov1960 потому что МТС достало, а пчел н..."


In [22]:
d = [tokenize(i) for i in data['text']] 
data_tok['tokenized'] = d

In [23]:
data_tok.head()

,label,text,tokenized
0,0,"@mkomov Максим, Вашем письмо мы получили. Наши...",mkomov максим вашем письмо мы получили наши со...
1,0,«Мегафон» стал владельцем 50% акций «Евросети»,мегафон стал владельцем 50 акций евросети
2,-1,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...,rt fuckkiev evakobb мтс россия прислала жителя...
3,1,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...,видео реклама со смехом мтс супер 0
4,-1,"@parfenov1960 потому что МТС достало, а пчел н...",parfenov1960 потому что мтс достало а пчел нен...


In [24]:
t = [TextBlob(i) for i in data_tok['tokenized']]
data_tok['TextBlob'] = t

s = [i.sentiment for i in data_tok['TextBlob']]
data_tok['sentiments'] = s

pred = []
for ind, sent in enumerate(data_tok['sentiments']):
    if sent.polarity < -0.25:
        pred.append(-1)
    elif sent.polarity > 0.25:
        pred.append(1)
    else:
        pred.append(0)

In [25]:
data_tok.head()

,label,text,tokenized,TextBlob,sentiments
0,0,"@mkomov Максим, Вашем письмо мы получили. Наши...",mkomov максим вашем письмо мы получили наши со...,"(m, k, o, m, o, v, , м, а, к, с, и, м, , в, ...","(0.0, 0.0)"
1,0,«Мегафон» стал владельцем 50% акций «Евросети»,мегафон стал владельцем 50 акций евросети,"(м, е, г, а, ф, о, н, , с, т, а, л, , в, л, ...","(0.0, 0.0)"
2,-1,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...,rt fuckkiev evakobb мтс россия прислала жителя...,"(r, t, , f, u, c, k, k, i, e, v, , e, v, a, ...","(0.0, 0.0)"
3,1,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...,видео реклама со смехом мтс супер 0,"(в, и, д, е, о, , р, е, к, л, а, м, а, , с, ...","(0.0, 0.0)"
4,-1,"@parfenov1960 потому что МТС достало, а пчел н...",parfenov1960 потому что мтс достало а пчел нен...,"(p, a, r, f, e, n, o, v, 1, 9, 6, 0, , п, о, ...","(0.0, 0.0)"


In [26]:
print(classification_report(data_tok['label'], data['pred']))

              precision    recall  f1-score   support

          -1       0.69      0.01      0.03      2388
           0       0.56      0.98      0.72      4590
           1       0.37      0.05      0.09      1230

   micro avg       0.56      0.56      0.56      8208
   macro avg       0.54      0.35      0.28      8208
weighted avg       0.57      0.56      0.42      8208



Лучше не стало. Попробуем встроенный в TextBlob классификатор

In [27]:
train = []


for ind, text in enumerate(train_data['text']):
    train.append(tuple([text, train_data['label'][ind]]))
    
test = []
for ind, text in enumerate(test_data['text']):
    test.append(tuple([text, test_data['label'][ind]]))

In [28]:
train[:5]

[('@mkomov Максим, Вашем письмо мы получили. Наши сотрудники свяжутся с Вами завтра и направят запрос инженерам для проверки. #билайн',
  0),
 ('«Мегафон» стал владельцем 50% акций «Евросети»', 0),
 ('RT @fuckkiev: “@EvaKobb: МТС Россия прислала жителям Херсонщины сообщения, в которых обозвала украинцев фашистами? http://t.co/RbSesXlOUZ” …',
  -1),
 ('ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехом МТС - Супер 0', 1),
 ('@parfenov1960 потому что МТС достало, а пчел ненавижу с детства, как и их мёд!',
  -1)]

In [29]:
test[:5]

[('RT vzglyad: По делу о работе МТС в Узбекистане США предложили заморозить 300 млн',
  -1),
 ('RT @kevinuyatukox: http://t.co/ljtrjq91v3 #Кредитные карты мегафон банка',
  0),
 ('#Оформить кредитную карту в банке мтс http://t.co/vv1B6PMWgH', 0),
 ('#Как перевести деньги с билайна на кредитную карту', 0),
 ('#Начальник отдела кредитного контроля оао мтс усачева н а', 0)]

In [31]:
len(test)

2054

In [30]:
from textblob.classifiers import NaiveBayesClassifier
from textblob.classifiers import DecisionTreeClassifier

Почему-то встроенные в textblob классификаторы оооооочень долго работают при большом количестве текстов. Поэтому я решила уменьшить количество путем уравнивания классов.

In [32]:
train = []
train_neg = 0
train_netr = 0

for ind, text in enumerate(train_data['text']):
    if train_data['label'][ind] == -1:
        if train_neg < 1230:
            train.append(tuple([text, train_data['label'][ind]]))
            train_neg+=1
    
    elif train_data['label'][ind] == 0:
        if train_netr < 1230:
            train.append(tuple([text, train_data['label'][ind]]))
            train_netr+=1
    
    else:
        train.append(tuple([text, train_data['label'][ind]]))

In [44]:
test = []
test_neg = 0
test_netr = 0

for ind, text in enumerate(test_data['text']):
    if test_data['label'][ind] == -1:
        if test_neg < 180:
            test.append(tuple([text, test_data['label'][ind]]))
            test_neg+=1
    
    elif test_data['label'][ind] == 0:
        if test_netr < 180:
            test.append(tuple([text, test_data['label'][ind]]))
            test_netr+=1
    
    else:
        test.append(tuple([text, test_data['label'][ind]]))

In [45]:
print(len(test))

540


In [46]:
print(len(train))

3690


In [40]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/sashamattini/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
%%time
cl = DecisionTreeClassifier(train)

CPU times: user 38.6 s, sys: 3.81 s, total: 42.4 s
Wall time: 42.4 s


In [49]:
%%time
cl.accuracy(test[:5])

KeyboardInterrupt: 

### Fail
Что мне удалось выяснить: 
- скорее всего, что-то сломалось в библиотеке, потому что на этапе прогона модели на тесте, она уходит в непрерывную работу на непонятное количество часов (комп с 8гб оперативы вообще довела до логов).
- от количество данных это не зависит, потому что я их уменьшала, а результат тот же

### Что еще умеет TextBlob?

In [58]:
ex = pd.DataFrame()
ex['text'] = copy(train_data['text'][:5])
t = [TextBlob(i) for i in ex['text']]
ex['TextBlob'] = t
ex

,text,TextBlob
0,"@mkomov Максим, Вашем письмо мы получили. Наши...","(@, m, k, o, m, o, v, , М, а, к, с, и, м, ,, ..."
1,«Мегафон» стал владельцем 50% акций «Евросети»,"(«, М, е, г, а, ф, о, н, », , с, т, а, л, , ..."
2,RT @fuckkiev: “@EvaKobb: МТС Россия прислала ж...,"(R, T, , @, f, u, c, k, k, i, e, v, :, , “, ..."
3,ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехо...,"(В, И, Д, Е, О, :, , h, t, t, p, :, /, /, t, ..."
4,"@parfenov1960 потому что МТС достало, а пчел н...","(@, p, a, r, f, e, n, o, v, 1, 9, 6, 0, , п, ..."


In [69]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     /home/sashamattini/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [64]:
for el in ex['TextBlob']:
    print(el.detect_language())

ru
ru
ru
ru
ru


Определять часть

In [66]:
for el in ex['TextBlob']:
    print(el.pos_tags)
    print('---')

[('@', 'JJ'), ('mkomov', 'NN'), ('Максим', 'NN'), ('Вашем', 'NNP'), ('письмо', 'NNP'), ('мы', 'NNP'), ('получили', 'NNP'), ('Наши', 'JJ'), ('сотрудники', 'NNP'), ('свяжутся', 'NNP'), ('с', 'NNP'), ('Вами', 'NNP'), ('завтра', 'NNP'), ('и', 'NNP'), ('направят', 'NNP'), ('запрос', 'NNP'), ('инженерам', 'NNP'), ('для', 'NNP'), ('проверки', 'NNP'), ('билайн', 'NN')]
---
[('«Мегафон»', 'JJ'), ('стал', 'NNP'), ('владельцем', 'NNP'), ('50', 'CD'), ('%', 'NN'), ('акций', 'NN'), ('«Евросети»', 'NN')]
---
[('RT', 'NNP'), ('@', 'NNP'), ('fuckkiev', 'NN'), ('“', 'JJ'), ('@', 'JJ'), ('EvaKobb', 'NN'), ('МТС', 'JJ'), ('Россия', 'NNP'), ('прислала', 'NNP'), ('жителям', 'NNP'), ('Херсонщины', 'NNP'), ('сообщения', 'NNP'), ('в', 'NNP'), ('которых', 'NNP'), ('обозвала', 'NNP'), ('украинцев', 'NNP'), ('фашистами', 'NNP'), ('http', 'NN'), ('//t.co/RbSesXlOUZ”', 'NN'), ('…', 'NN')]
---
[('ВИДЕО', 'NN'), ('http', 'NN'), ('//t.co/PSMLAhR4fI', 'JJ'), ('Реклама', 'NNP'), ('со', 'NNP'), ('смехом', 'NNP'), ('МТС'

Доставать именные фразы

In [70]:
for el in ex['TextBlob']:
    print(el.noun_phrases)
    print('---')

['@ mkomov максим', 'вашем письмо мы получили', 'наши сотрудники свяжутся с вами завтра и направят запрос инженерам для проверки', '# билайн']
---
['«мегафон» стал владельцем', '% акций «евросети»']
---
['rt', '@ fuckkiev', '“ @', 'evakobb', 'мтс россия прислала жителям херсонщины сообщения', 'в которых обозвала украинцев фашистами', '//t.co/rbsesxlouz” …']
---
['//t.co/psmlahr4fi реклама со смехом мтс']
---
['@ parfenov1960 потому что мтс достало', 'а пчел ненавижу с детства', 'как и их мёд']
---


In [72]:
from textblob import Word

In [73]:
for el in ex['TextBlob']:
    print(el.words)
    print('---')

['mkomov', 'Максим', 'Вашем', 'письмо', 'мы', 'получили', 'Наши', 'сотрудники', 'свяжутся', 'с', 'Вами', 'завтра', 'и', 'направят', 'запрос', 'инженерам', 'для', 'проверки', 'билайн']
---
['«Мегафон»', 'стал', 'владельцем', '50', 'акций', '«Евросети»']
---
['RT', 'fuckkiev', '“', 'EvaKobb', 'МТС', 'Россия', 'прислала', 'жителям', 'Херсонщины', 'сообщения', 'в', 'которых', 'обозвала', 'украинцев', 'фашистами', 'http', 't.co/RbSesXlOUZ”', '…']
---
['ВИДЕО', 'http', 't.co/PSMLAhR4fI', 'Реклама', 'со', 'смехом', 'МТС', 'Супер', '0']
---
['parfenov1960', 'потому', 'что', 'МТС', 'достало', 'а', 'пчел', 'ненавижу', 'с', 'детства', 'как', 'и', 'их', 'мёд']
---


In [74]:
for el in ex['TextBlob']:
    print(el.words[3].singularize())
    print(el.words[4].pluralize())
    print('---')

письмо
мыs
---
50
акцийs
---
EvaKobb
МТСs
---
Реклама
соs
---
МТС
досталоs
---


In [76]:
for el in ex['TextBlob']:
    print(el)
    print(el.translate(to='en'))
    print('---')

@mkomov Максим, Вашем письмо мы получили. Наши сотрудники свяжутся с Вами завтра и направят запрос инженерам для проверки. #билайн
@mkomov Maxim, we received your letter. Our staff will contact you tomorrow and send a request to engineers for verification. # Beeline
---
«Мегафон» стал владельцем 50% акций «Евросети»
MegaFon became the owner of 50% of Euroset shares
---
RT @fuckkiev: “@EvaKobb: МТС Россия прислала жителям Херсонщины сообщения, в которых обозвала украинцев фашистами? http://t.co/RbSesXlOUZ” …
RT @fuckkiev: “@EvaKobb: MTS Russia sent messages to residents of Kherson region, in which she called Ukrainians fascists? http://t.co/RbSesXlOUZ ”...
---
ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехом МТС - Супер 0
VIDEO: http://t.co/PSMLAhR4fI Advertising with laughter MTS - Super 0
---
@parfenov1960 потому что МТС достало, а пчел ненавижу с детства, как и их мёд!
@ parfenov1960 because MTS has got enough, but I hate bees since childhood, like their honey!
---


In [77]:
ch = []
for el in ex['TextBlob']:
    print(el)
    ch.append(el.translate(to='zh-CN'))
    print(el.translate(to='zh-CN'))
    print('---')
ex['zh-CN'] = ch

@mkomov Максим, Вашем письмо мы получили. Наши сотрудники свяжутся с Вами завтра и направят запрос инженерам для проверки. #билайн
@mkomov Maxim，我们收到了你的来信。我们的工作人员明天将与您联系并向工程师发送请求以进行验证。 ＃直线
---
«Мегафон» стал владельцем 50% акций «Евросети»
MegaFon成为Euroset 50％股份的所有者
---
RT @fuckkiev: “@EvaKobb: МТС Россия прислала жителям Херсонщины сообщения, в которых обозвала украинцев фашистами? http://t.co/RbSesXlOUZ” …
RT @fuckkiev：“@EvaKobb：MTS俄罗斯向赫尔松地区的居民发送信息，她称乌克兰人为法西斯主义者？ http://t.co/RbSesXlOUZ“......
---
ВИДЕО: http://t.co/PSMLAhR4fI Реклама со смехом МТС - Супер 0
视频：http：//t.co/PSMLAhR4fI广告笑声MTS  - 超级0
---
@parfenov1960 потому что МТС достало, а пчел ненавижу с детства, как и их мёд!
@ parfenov1960因为MTS已经足够了，但我从小就讨厌蜜蜂，就像他们的亲爱的！
---


In [78]:
for el in ex['zh-CN']:
    print(el)
    print(el.translate(from_lang="zh-CN", to='ru'))
    print('---')

@mkomov Maxim，我们收到了你的来信。我们的工作人员明天将与您联系并向工程师发送请求以进行验证。 ＃直线
@mkomov Максим, мы получили твое письмо. Наши сотрудники свяжутся с вами завтра и отправят запрос инженеру для проверки. # Straight
---
MegaFon成为Euroset 50％股份的所有者
МегаФон становится владельцем 50% Евросети
---
RT @fuckkiev：“@EvaKobb：MTS俄罗斯向赫尔松地区的居民发送信息，她称乌克兰人为法西斯主义者？ http://t.co/RbSesXlOUZ“......
RT @fuckkiev: «@EvaKobb: МТС Россия посылает информацию жителям Херсонщины. Называет ли она украинцев фашистами? Http://t.co/RbSesXlOUZ» ...
---
视频：http：//t.co/PSMLAhR4fI广告笑声MTS  - 超级0
Видео: http://t.co/PSMLAhR4fI Реклама Смех МТС - Супер 0
---
@ parfenov1960因为MTS已经足够了，但我从小就讨厌蜜蜂，就像他们的亲爱的！
@ parfenov1960 потому что МТС достаточно, но я ненавижу пчел с раннего возраста, как и их дорогая
---


In [79]:
for el in ex['zh-CN']:
    print(el)
    print(el.translate(from_lang="zh-CN", to='en'))
    print('---')

@mkomov Maxim，我们收到了你的来信。我们的工作人员明天将与您联系并向工程师发送请求以进行验证。 ＃直线
@mkomov Maxim, we have received your letter. Our staff will contact you tomorrow and send a request to the engineer for verification. #straight line
---
MegaFon成为Euroset 50％股份的所有者
MegaFon becomes the owner of 50% of Euroset
---
RT @fuckkiev：“@EvaKobb：MTS俄罗斯向赫尔松地区的居民发送信息，她称乌克兰人为法西斯主义者？ http://t.co/RbSesXlOUZ“......
RT @fuckkiev: "@EvaKobb: MTS Russia sends information to residents of the Kherson region. Does she call the Ukrainians a fascist? http://t.co/RbSesXlOUZ"...
---
视频：http：//t.co/PSMLAhR4fI广告笑声MTS  - 超级0
Video: http://t.co/PSMLAhR4fI Advertising Laughter MTS - Super 0
---
@ parfenov1960因为MTS已经足够了，但我从小就讨厌蜜蜂，就像他们的亲爱的！
@parfenov1960 because MTS is enough, but I hate bees from an early age, just like their dear!
---


### Итог:

Плюсы:
- классификатор
- анализатор натроений
- плюрализатор
- исправление написания
- переводчик, который даже не плох.

Минусы:
- позиционируется как подходящая для русского, а на самом деле не совсем.
- возможно, именно из-за этого у меня классификатор работал плохо.

К сожалению, у меня не было данных для того, чтобы проверить его работу с английским. Возможно, классификатор показал бы себя лучше.